In [660]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
df = pd.read_csv('ML_Data/DJI_pos.csv')


#BTC_neg.csv
#BTC_pos.csv
#BTC_vader.csv

#DJI_neg.csv
#DJI_pos.csv
#DJI_vader.csv

#TSLA_neg.csv
#TSLA_pos.csv
#TSLA_vader.csv

df = df.drop(['Unnamed: 0'], axis=1)

train = df.iloc[50:]
test = df.iloc[:50]


df.head()

,date,negatives,positives,ratio,negatives (t-1),positives (t-1),ratio (t-1),negatives (t-2),positives (t-2),ratio (t-2),...,negatives (t-5),positives (t-5),ratio (t-5),negatives (t-6),positives (t-6),ratio (t-6),negatives (t-7),positives (t-7),ratio (t-7),label
0,2019-05-03,10.0,5.0,0.333333,15.0,8.0,0.347826,13.0,10.0,0.434783,...,0.0,0.0,0.000000,0.0,0.0,0.000000,13.0,10.0,0.434783,0
1,2019-05-02,15.0,8.0,0.347826,13.0,10.0,0.434783,5.0,3.0,0.375000,...,0.0,0.0,0.000000,13.0,10.0,0.434783,18.0,7.0,0.280000,1
2,2019-05-01,13.0,10.0,0.434783,5.0,3.0,0.375000,4.0,3.0,0.428571,...,13.0,10.0,0.434783,18.0,7.0,0.280000,7.0,5.0,0.416667,0
3,2019-04-30,5.0,3.0,0.375000,4.0,3.0,0.428571,0.0,0.0,0.000000,...,18.0,7.0,0.280000,7.0,5.0,0.416667,16.0,5.0,0.238095,0
4,2019-04-29,4.0,3.0,0.428571,0.0,0.0,0.000000,0.0,0.0,0.000000,...,7.0,5.0,0.416667,16.0,5.0,0.238095,7.0,4.0,0.363636,1


# Visualize data

In [635]:
#df.head()

In [636]:
#df['label'].value_counts().plot(kind='bar')

# Normalize  data

In [655]:

X_train = train.drop(['label','date'], axis=1)
X_train = X_train.values
y_train = train['label']

X_test = test.drop(['label','date'], axis=1)
X_test = X_test.values
y_test = test['label']

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [656]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(286, 24)
(50, 24)
(286,)
(50,)


In [604]:
pipeline = Pipeline([
    ('normalizer', StandardScaler()), #Step1 - normalize data
    ('clf', LogisticRegression()) #step2 - classifier 
])
pipeline.steps

[('normalizer', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('clf',
  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='warn',
            n_jobs=None, penalty='l2', random_state=None, solver='warn',
            tol=0.0001, verbose=0, warm_start=False))]

# Split data

In [605]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(228, 24)
(58, 24)
(228,)
(58,)


# Estimate classifiers

In [613]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

clfs = []
#clfs.append(LogisticRegression(solver='lbfgs',C=0.5570175438596491))
clfs.append(SVC())
#C=0.03,gamma=0.001,kernel='linear'
#clfs.append(GaussianNB())
#clfs.append(KNeighborsClassifier())
#clfs.append(RandomForestClassifier(n_estimators=20))

for classifier in clfs:
    pipeline.set_params(clf = classifier)
    scores = cross_validate(pipeline, X_train, y_train, cv=10)
    
    print('---------------------------------')
    print(str(classifier))
    print('-----------------------------------')
    for key, values in scores.items():
            print(key,' mean ', values.mean())
            #print(key,' std ', values.std())'''

---------------------------------
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)
-----------------------------------
fit_time  mean  0.005284953117370606
score_time  mean  0.0009465217590332031
test_score  mean  0.5045454545454546
train_score  mean  0.821631541558134


In [607]:
from sklearn.model_selection import GridSearchCV
pipeline.set_params(clf = SVC())
pipeline.steps

[('normalizer', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('clf', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False))]

In [608]:
cv_grid = GridSearchCV(pipeline, param_grid = {
    #'clf__solver' : ['lbfgs','newton-cg'],
    'clf__kernel' : ['linear', 'rbf'],
    #'clf__C' : np.linspace(0.1,1.2,12),
    'clf__C' : np.linspace(0.01,1.0,100),
    'clf__gamma' : [0.001, 0.01, 0.1, 1,'auto_deprecated']
},cv=3)

#solver='lbfgs'

cv_grid.fit(X_train, y_train)



GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('normalizer', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'clf__kernel': ['linear', 'rbf'], 'clf__C': array([0.01, 0.02, ..., 0.99, 1.  ]), 'clf__gamma': [0.001, 0.01, 0.1, 1, 'auto_deprecated']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [609]:
cv_grid.best_params_

{'clf__C': 0.02, 'clf__gamma': 0.001, 'clf__kernel': 'linear'}

In [610]:
cv_grid.best_estimator_

Pipeline(memory=None,
     steps=[('normalizer', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', SVC(C=0.02, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [611]:
cv_grid.best_score_

0.5701754385964912

In [612]:
y_predict = cv_grid.predict(X_test)
accuracy = accuracy_score(y_test,y_predict)
print('Accuracy of the best classifier after CV is %.5f%%' % (accuracy*100))

Accuracy of the best classifier after CV is 39.65517%


## Predict 50

In [673]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix


lr = LogisticRegression(solver='lbfgs',C=0.472, random_state = 1)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
print(y_pred)
list_pred = list(zip(X_test, y_pred))
#print(list_[0])


[0 0 1 0 1 0 1 1 1 0 1 1 1 1 0 1 1 0 0 0 0 1 1 1 1 0 1 1 1 0 0 1 1 1 1 0 1
 1 1 0 1 1 1 0 0 0 0 1 1 0]


In [678]:
test.head()

,date,negatives,positives,ratio,negatives (t-1),positives (t-1),ratio (t-1),negatives (t-2),positives (t-2),ratio (t-2),...,negatives (t-5),positives (t-5),ratio (t-5),negatives (t-6),positives (t-6),ratio (t-6),negatives (t-7),positives (t-7),ratio (t-7),label
0,2019-05-03,10.0,5.0,0.333333,15.0,8.0,0.347826,13.0,10.0,0.434783,...,0.0,0.0,0.000000,0.0,0.0,0.000000,13.0,10.0,0.434783,0
1,2019-05-02,15.0,8.0,0.347826,13.0,10.0,0.434783,5.0,3.0,0.375000,...,0.0,0.0,0.000000,13.0,10.0,0.434783,18.0,7.0,0.280000,1
2,2019-05-01,13.0,10.0,0.434783,5.0,3.0,0.375000,4.0,3.0,0.428571,...,13.0,10.0,0.434783,18.0,7.0,0.280000,7.0,5.0,0.416667,0
3,2019-04-30,5.0,3.0,0.375000,4.0,3.0,0.428571,0.0,0.0,0.000000,...,18.0,7.0,0.280000,7.0,5.0,0.416667,16.0,5.0,0.238095,0
4,2019-04-29,4.0,3.0,0.428571,0.0,0.0,0.000000,0.0,0.0,0.000000,...,7.0,5.0,0.416667,16.0,5.0,0.238095,7.0,4.0,0.363636,1


In [679]:
df_done = test.drop(['label'], axis=1)
df_done['Label'] = pd.Series(y_pred)
df_done.head(50)

,date,negatives,positives,ratio,negatives (t-1),positives (t-1),ratio (t-1),negatives (t-2),positives (t-2),ratio (t-2),...,negatives (t-5),positives (t-5),ratio (t-5),negatives (t-6),positives (t-6),ratio (t-6),negatives (t-7),positives (t-7),ratio (t-7),Label
0,2019-05-03,10.0,5.0,0.333333,15.0,8.0,0.347826,13.0,10.0,0.434783,...,0.0,0.0,0.000000,0.0,0.0,0.000000,13.0,10.0,0.434783,0
1,2019-05-02,15.0,8.0,0.347826,13.0,10.0,0.434783,5.0,3.0,0.375000,...,0.0,0.0,0.000000,13.0,10.0,0.434783,18.0,7.0,0.280000,0
2,2019-05-01,13.0,10.0,0.434783,5.0,3.0,0.375000,4.0,3.0,0.428571,...,13.0,10.0,0.434783,18.0,7.0,0.280000,7.0,5.0,0.416667,1
3,2019-04-30,5.0,3.0,0.375000,4.0,3.0,0.428571,0.0,0.0,0.000000,...,18.0,7.0,0.280000,7.0,5.0,0.416667,16.0,5.0,0.238095,0
4,2019-04-29,4.0,3.0,0.428571,0.0,0.0,0.000000,0.0,0.0,0.000000,...,7.0,5.0,0.416667,16.0,5.0,0.238095,7.0,4.0,0.363636,1
5,2019-04-26,13.0,10.0,0.434783,18.0,7.0,0.280000,7.0,5.0,0.416667,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0
6,2019-04-25,18.0,7.0,0.280000,7.0,5.0,0.416667,16.0,5.0,0.238095,...,0.0,0.0,0.000000,0.0,0.0,0.000000,6.0,16.0,0.727273,1
7,2019-04-24,7.0,5.0,0.416667,16.0,5.0,0.238095,7.0,4.0,0.363636,...,0.0,0.0,0.000000,6.0,16.0,0.727273,9.0,4.0,0.307692,1
8,2019-04-23,16.0,5.0,0.238095,7.0,4.0,0.363636,0.0,0.0,0.000000,...,6.0,16.0,0.727273,9.0,4.0,0.307692,15.0,6.0,0.285714,1
9,2019-04-22,7.0,4.0,0.363636,0.0,0.0,0.000000,0.0,0.0,0.000000,...,9.0,4.0,0.307692,15.0,6.0,0.285714,11.0,5.0,0.312500,0


# Output

In [680]:
new_test_df.to_csv('ML_Data/DJI_pos_50_labeled.csv')

NameError: name 'new_test_df' is not defined

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)